In [1]:
import os
import numpy as np
import tensorflow as tf
import IPython.display as ipd

from scipy.io.wavfile import read

from model.vq_vae import VQ_VAE
from model.wavenet import WaveNet
from vq_vae_trainer import VQ_VAE_Trainer
from wavenet_trainer import WaveNet_Trainer
from model.conv_encoder import ConvEncoder, ConvDecoder

In [2]:
############################################################
# Current VQ-VAE implementation here
# - Input audio
# ---> Conv Encoder (Downsample)
# ---> VQ (Latent space)
# ---> Conv Decoder (Upsample)
# ---> WaveNet (Generate)
# - Output audio
############################################################
wavenet = WaveNet()
#conv_encoder = ConvEncoder()
#vq = VQ_VAE()

Model: "WaveNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 16000, 1)]   0                                            
__________________________________________________________________________________________________
Initial-Causal-Convolution (Con (None, 16000, 32)    1056        input_1[0][0]                    
__________________________________________________________________________________________________
Layer0-Conv-Filter (Conv1D)     (None, 16000, 32)    4128        Initial-Causal-Convolution[0][0] 
__________________________________________________________________________________________________
Layer0-Conv-Gate (Conv1D)       (None, 16000, 32)    4128        Initial-Causal-Convolution[0][0] 
____________________________________________________________________________________________

In [3]:
DATA_PATH = r"/home/rithomas/data/IDMT-SMT-GUITAR_V2/dataset1/Fender_Strat_Clean_Neck_SC/audio"
p = os.path.join(DATA_PATH, 'G53-40100-1111-00001.wav')
print(read(p))

(44100, array([6, 5, 4, ..., 0, 0, 0], dtype=int16))


In [4]:
DATA_PATH='/home/rithomas/data/test_preprocessed.npy'
raw_dataset = np.load(DATA_PATH)
print(raw_dataset)
## Dummy dataset to test architecture
def generator():
    for x in raw_dataset:
        yield tf.expand_dims(x, axis=-1)
    
ds = tf.data.Dataset.from_generator(generator, output_types=tf.int32)
ds = ds.batch(4).shuffle(100)
print(ds)

#for x in ds:
#    wav = tf.squeeze(x[0,...]).numpy()
#    print(wav)
#    ipd.display(ipd.Audio(wav, rate=16000))

TRAINING_CONFIGS = {
    'model_path': '/home/rithomas/cache/test_model',
    'learning_rate': 0.0001,
    'num_epochs': 100,
    'print_every': 1000,
    'save_every': 1000,
}
trainer = WaveNet_Trainer(wavenet, TRAINING_CONFIGS)
trainer.train(ds)

[[ 3.95994371e-01 -4.98146801e+00 -3.22002837e+00 ...  1.55264104e+03
   1.62975531e+03  1.42057785e+03]
 [ 1.36941664e+01 -1.30742343e+02 -2.00229754e+02 ... -2.19710113e+02
  -2.82750540e+02 -2.77170418e+02]
 [ 3.76769611e+00  1.61853977e+00  2.54790664e+00 ... -5.16470054e+01
   1.53192898e+02  4.84610475e+02]
 ...
 [-2.19775480e+02 -6.77564272e+01 -5.47918268e+01 ... -2.32238037e+03
  -1.87986713e+03 -2.40932383e+03]
 [-3.42904719e+02 -4.72849466e+02 -7.36682740e+02 ...  2.31002633e+00
   1.40627736e+00  2.54866870e+00]
 [-6.68388671e+02 -2.37689511e+02  1.42882113e+01 ... -5.12406201e+03
  -5.31629068e+03 -4.40499748e+03]]
<ShuffleDataset shapes: <unknown>, types: tf.int32>
Initializing from scratch.
tf.Tensor(5.545245, shape=(), dtype=float32)
PREDICTED:


ORIGINAL (Mu-law):


ORIGINAL:


KeyError: 'save_every'